In [1]:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from instagrapi import Client

In [39]:
data = pd.read_csv('insta_flower.csv')

## 1. 데이터 탐색

### 결측치 파악

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
username         1000 non-null object
location         482 non-null object
lat              481 non-null float64
lng              481 non-null float64
comment_count    1000 non-null int64
like_count       1000 non-null int64
caption_text     1000 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 54.8+ KB


location, lat, lng 518개 결측치

### 위치정보가 나오지 않은 행들의 계정정보 확인

In [4]:
data[data['location'].isnull()]['username'].value_counts()

blooming__bouquet       158
cloverflower_hongdae    144
rikaakim                 95
herafleur                54
mondaymarket_            21
for.rest.official        17
tteul.jung_              11
ppink_flower              7
della_gem                 3
hwayoilflowers            2
dbel_yes                  1
gongbangworld             1
iamo1991_official         1
fawn_seoul                1
studio_darluz             1
goeun_gjw                 1
Name: username, dtype: int64

상위 5개 먼저 확인

- blooing__bouquet, cloverflower_hongdae : 대부분 Null값이지만 위치정보 나와있는 행 존재
- rikaakim, herafleur, mondaymarket_ : 클래스 운영도 하는것같은데 위치정보 없음


## 2. 텍스트 분석

### 게시글에서 해시태그만 추출

In [6]:
import re

In [7]:
for i in range(len(data)):
    data.loc[i,'hashtag'] = ' '.join(re.findall(r'#\w+',data.loc[i,'caption_text']))

### 해시태그에 '광고', '홍보' 문구가 들어가있는 경우의 인덱스 추출

In [8]:
ad_index = []

for i in range(len(data)):
    if '광고' in data.loc[i,'hashtag'] or '홍보' in data.loc[i,'hashtag'] or '대행' in data.loc[i,'hashtag']:
        ad_index.append(i)

In [9]:
ad_index

[272, 665, 777]

=> #광고촬영 이라는 해시태그가 들어있어서 걸러짐 => 삭제하지는 않기로함

### 해시태그 이름별로 숫자 카운트 => 딕셔너리에 저장

In [10]:
hashtag_count = {}

In [11]:
for i in range(len(data)):
    hash_split = (data.loc[i,'hashtag']).split(' ')
    
    for k in hash_split:
        
        if k not in hashtag_count.keys():
            hashtag_count[k] = 1
        else:
            hashtag_count[k] += 1

In [12]:
hashtag_count

{'#홍대꽃집': 918,
 '#홍대꽃집추천': 20,
 '#서교동꽃집': 1003,
 '#망원동꽃집': 119,
 '#망원꽃집': 129,
 '#강남꽃집': 72,
 '#상수꽃집': 217,
 '#서강대꽃집': 46,
 '#홍대꽃배달': 61,
 '#서울꽃배달': 9,
 '#합정꽃집': 363,
 '#마포꽃집': 309,
 '#합정역꽃집': 317,
 '#연남동꽃집': 758,
 '#신촌꽃집': 364,
 '#홍대원데이클래스': 205,
 '#커플클래스': 9,
 '#이색데이트추천': 1,
 '#연예인꽃집': 1,
 '#연예인꽃다발': 120,
 '#카이정헤어': 1,
 '#서교동미용실': 1,
 '#합정역미용실': 1,
 '#공덕미용실': 1,
 '#합정역카페': 1,
 '#중단발스타일': 1,
 '#연남동빵집': 1,
 '#서교동카페': 1,
 '#동교동카페': 1,
 '#그날의무드고결': 1,
 '#가좌동꽃집': 103,
 '#상암동꽃집': 256,
 '#인천꽃집': 161,
 '#일산꽃집': 155,
 '#용산꽃집': 184,
 '#floraldesign': 204,
 '#flowerartist': 172,
 '#웨딩부케': 264,
 '#플로리스트': 261,
 '#프로포즈': 213,
 '#본식부케': 227,
 '#럽스타그램': 237,
 '#결혼식부케': 226,
 '#야외부케': 189,
 '#프로포즈꽃다발': 479,
 '#꽃다발': 356,
 '#꽃바구니': 314,
 '#플라워박스': 180,
 '#수국꽃다발': 31,
 '#수국꽃바구니': 5,
 '#생일선물': 16,
 '#용돈박스': 60,
 '#수국': 5,
 '#마리앙로즈': 234,
 '#마리앙로즈꽃다발': 136,
 '#장미부케': 32,
 '#장미꽃다발': 90,
 '#수입장미꽃다발': 8,
 '#기념일': 17,
 '#로즈데이': 11,
 '#스승의날': 15,
 '#카네이션꽃다발': 16,
 '#꽃다발선물': 23,
 '#뜰정이벤트': 1,
 '#칠링백': 1,
 '#가

In [29]:
len(hashtag_count)

1413

#### 1번밖에 나오지 않은 해시태그는 제외

In [13]:
hashtag_count = {key: value for key, value in hashtag_count.items() if value >=2}

In [14]:
hashtag_count

{'#홍대꽃집': 918,
 '#홍대꽃집추천': 20,
 '#서교동꽃집': 1003,
 '#망원동꽃집': 119,
 '#망원꽃집': 129,
 '#강남꽃집': 72,
 '#상수꽃집': 217,
 '#서강대꽃집': 46,
 '#홍대꽃배달': 61,
 '#서울꽃배달': 9,
 '#합정꽃집': 363,
 '#마포꽃집': 309,
 '#합정역꽃집': 317,
 '#연남동꽃집': 758,
 '#신촌꽃집': 364,
 '#홍대원데이클래스': 205,
 '#커플클래스': 9,
 '#연예인꽃다발': 120,
 '#가좌동꽃집': 103,
 '#상암동꽃집': 256,
 '#인천꽃집': 161,
 '#일산꽃집': 155,
 '#용산꽃집': 184,
 '#floraldesign': 204,
 '#flowerartist': 172,
 '#웨딩부케': 264,
 '#플로리스트': 261,
 '#프로포즈': 213,
 '#본식부케': 227,
 '#럽스타그램': 237,
 '#결혼식부케': 226,
 '#야외부케': 189,
 '#프로포즈꽃다발': 479,
 '#꽃다발': 356,
 '#꽃바구니': 314,
 '#플라워박스': 180,
 '#수국꽃다발': 31,
 '#수국꽃바구니': 5,
 '#생일선물': 16,
 '#용돈박스': 60,
 '#수국': 5,
 '#마리앙로즈': 234,
 '#마리앙로즈꽃다발': 136,
 '#장미부케': 32,
 '#장미꽃다발': 90,
 '#수입장미꽃다발': 8,
 '#기념일': 17,
 '#로즈데이': 11,
 '#스승의날': 15,
 '#카네이션꽃다발': 16,
 '#꽃다발선물': 23,
 '#가을꽃다발': 3,
 '#블루밍부케창업반': 127,
 '#블루밍부케꽃다발': 96,
 '#플로리스트학원': 248,
 '#플라워레슨창업반': 248,
 '#플로리스트': 248,
 '#꽃다발': 248,
 '#웨딩부케': 248,
 '#창전동꽃집': 248,
 '#플라워클래스

In [32]:
len(hashtag_count)

734

### 위치정보 카운트해 딕셔너리로 저장

In [15]:
location_count = {}

In [16]:
for i in range(len(data)):
    location = data.loc[i,'location']
    if location not in location_count:
        location_count[location] = 1
    else:
        location_count[location] += 1

In [17]:
location_count

{'서교동 332-20 아도니스플라워': 1,
 '홍대입구역 주변': 12,
 '카이정헤어': 1,
 '마리앙로즈 mariangrose': 241,
 nan: 518,
 '홍대': 1,
 '서교동': 6,
 '뜰,정': 1,
 '합정동 샤론꽃집': 42,
 '블루밍부케플라워샵': 88,
 '노적봉인공폭포공원': 1,
 'Hapjeong , Soeul': 5,
 '한강신도시 라베니체': 1,
 'Mecenatpolis Mall': 2,
 '합정 어딘가': 1,
 '이아모-iamo': 3,
 'Seoul, Korea': 1,
 '합정역': 2,
 '토끼풀꽃': 29,
 '홍대 아만티호텔': 15,
 '홍대입구근처': 2,
 '신세계백화점 영등포점': 1,
 '꽃집화요일': 19,
 '열두달, 꽃': 1,
 '마포구 성산동': 1,
 '샐리인더플라워 Sally in the flower': 3,
 'Hongik University Station': 1,
 '서소문로': 1}

In [36]:
len(location_count)

28

#### 위치에 1번밖에 나오지 않은 곳은 제외

In [18]:
location_count = {key: value for key, value in location_count.items() if value >= 2}

In [19]:
len(location_count)

15

In [20]:
location_count

{'홍대입구역 주변': 12,
 '마리앙로즈 mariangrose': 241,
 nan: 518,
 '서교동': 6,
 '합정동 샤론꽃집': 42,
 '블루밍부케플라워샵': 88,
 'Hapjeong , Soeul': 5,
 'Mecenatpolis Mall': 2,
 '이아모-iamo': 3,
 '합정역': 2,
 '토끼풀꽃': 29,
 '홍대 아만티호텔': 15,
 '홍대입구근처': 2,
 '꽃집화요일': 19,
 '샐리인더플라워 Sally in the flower': 3}

- 마리앙로즈 , 블루밍부케플라워샵, 합정동 샤론꽃집, 토끼풀꽃, 꽃집화요일, 이아모, 샐리인더플라워 => 위치정보 있는지 확인해보기

In [21]:
(data[data['location'].isin(['마리앙로즈 mariangrose','블루밍부케플라워샵','합정동 샤론꽃집','꽃집화요일','이아모-iamo','샐리인더플라워 Sally in the flower'])]).info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 4 to 999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   username       396 non-null    object 
 1   location       396 non-null    object 
 2   lat            396 non-null    float64
 3   lng            396 non-null    float64
 4   comment_count  396 non-null    int64  
 5   like_count     396 non-null    int64  
 6   caption_text   396 non-null    object 
 7   hashtag        396 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 27.8+ KB


- 확인결과, 모두 위치정보가 있음

### 위치로 많이태그된 곳의 username 조사

- 위치로 많이 태그되긴했지만, 이아모 제외 모두 네일운영자가 자신의 샵을 직접태그

In [22]:
data[data['location'] == '마리앙로즈 mariangrose']['username'].value_counts()

mariang_rose_    241
Name: username, dtype: int64

In [23]:
data[data['location'] == '블루밍부케플라워샵']['username'].value_counts()

blooming__bouquet    88
Name: username, dtype: int64

In [24]:
data[data['location'] == '합정동 샤론꽃집']['username'].value_counts()

sharon_flower1    42
Name: username, dtype: int64

In [25]:
data[data['location'] == '꽃집화요일']['username'].value_counts()

hwayoilflowers    19
Name: username, dtype: int64

In [26]:
data[data['location'] == '이아모-iamo']['username'].value_counts()

iamo1991_official    2
forroom_byhee        1
Name: username, dtype: int64

In [27]:
data[data['location'] == '샐리인더플라워 Sally in the flower']['username'].value_counts()

sallyintheflower    3
Name: username, dtype: int64

### 위치 빈도 높은곳들을 그룹으로 집계

In [28]:
popular_flower = data[data['location'].isin(['마리앙로즈 mariangrose','블루밍부케플라워샵','합정동 샤론꽃집','꽃집화요일','이아모-iamo','샐리인더플라워 Sally in the flower'])]

In [29]:
popular_flower.groupby(['location','username']).mean()

lat         lng  \
location                    username                                   
꽃집화요일                       hwayoilflowers     37.555967  126.917613   
마리앙로즈 mariangrose           mariang_rose_      37.515020  127.016480   
블루밍부케플라워샵                   blooming__bouquet  37.555565  126.930364   
샐리인더플라워 Sally in the flower sallyintheflower   37.556875  126.910259   
이아모-iamo                    forroom_byhee      37.549980  126.913764   
                            iamo1991_official  37.549980  126.913764   
합정동 샤론꽃집                    sharon_flower1     37.568900  126.977000   

                                               comment_count  like_count  
location                    username                                      
꽃집화요일                       hwayoilflowers          1.315789  128.947368  
마리앙로즈 mariangrose           mariang_rose_           0.580913  125.033195  
블루밍부케플라워샵                   blooming__bouquet       1.534091  340.261364  
샐리인더플라워 Sally in the flower sallyintheflower        2.333333  118.333333  
이아모-iamo                    forroom_byhee           1.000000  254.000000  
                            iamo1991_official       0.500000  127.500000  
합정동 샤론꽃집                    sharon_flower1          0.809524  159.666667

- 대부분 좋아요가 많이 찍히기는 함

#### 위 업체들이 해시태그로 있는지 확인

In [30]:
loc_shop = ['#마리앙로즈','#블루밍부케플라워샵','#샤론꽃집','#꽃집화요일','#이아모','#샐리인더플라워']

In [31]:
for i in loc_shop:
    print(i)
    
    if i in hashtag_count.keys():
        print(hashtag_count[i])
    else:
        print(0)

#마리앙로즈
234
#블루밍부케플라워샵
4
#샤론꽃집
14
#꽃집화요일
7
#이아모
0
#샐리인더플라워
3


### 해시태그 언급 횟수로 정렬해 핫플확인해보기

In [32]:
# hashtag_count 딕셔너리 정렬
hashtag_count_sorted= sorted(hashtag_count.items(), reverse=True, key=lambda item: item[1])



In [33]:
hashtag_count_sorted

[('#서교동꽃집', 1003),
 ('#홍대꽃집', 918),
 ('#연남동꽃집', 758),
 ('#프로포즈꽃다발', 479),
 ('#신촌꽃집', 364),
 ('#합정꽃집', 363),
 ('#꽃다발', 356),
 ('#합정역꽃집', 317),
 ('#꽃바구니', 314),
 ('#마포꽃집', 309),
 ('#플라워레슨', 296),
 ('#웨딩부케', 264),
 ('#플로리스트', 261),
 ('#상암동꽃집', 256),
 ('#플로리스트학원', 248),
 ('#플라워레슨창업반', 248),
 ('#플로리스트', 248),
 ('#꽃다발', 248),
 ('#웨딩부케', 248),
 ('#창전동꽃집', 248),
 ('#플라워클래스', 248),
 ('#서울플라워레슨', 248),
 ('#꽃다발포장법', 248),
 ('#플라워레슨', 248),
 ('#경의선숲길꽃집', 248),
 ('#경의선책거리', 244),
 ('#경의선책거리꽃집', 244),
 ('#럽스타그램', 237),
 ('#홍대플라워레슨', 235),
 ('#마리앙로즈', 234),
 ('#플라워클래스', 231),
 ('#본식부케', 227),
 ('#결혼식부케', 226),
 ('#상수꽃집', 217),
 ('#마포구청꽃집', 214),
 ('#프로포즈', 213),
 ('#홍대데이트', 206),
 ('#홍대원데이클래스', 205),
 ('#floraldesign', 204),
 ('#신촌숲아이파크', 199),
 ('#신촌숲아이파크꽃집', 199),
 ('#홍대입구역꽃집', 193),
 ('#야외부케', 189),
 ('#용산꽃집', 184),
 ('#플라워박스', 180),
 ('#flowerartist', 172),
 ('#인천꽃집', 161),
 ('#이대꽃집', 160),
 ('#일산꽃집', 155),
 ('#일상', 150),
 ('#연대꽃집', 

- 대부분 @@@동네일 태그가 많음
- 상호 이름으로 추정되는 곳 : 마리앙로즈(234), 블루밍,헤라플레르(54),토끼풀꽃,샤론,이아모, 

In [34]:
# 비마이문네일, 이솝네일 살펴보기

mariang = data[data['hashtag'].str.contains('#마리앙로즈') == True]
blueming = data[data['hashtag'].str.contains('#블루밍') == True]
hera = data[data['hashtag'].str.contains('#헤라플레르') == True]
rabbit = data[data['hashtag'].str.contains('#토끼풀꽃') == True]
sharon = data[data['hashtag'].str.contains('#샤론') == True]
iamo = data[data['hashtag'].str.contains('#이아모') == True]


In [35]:
hash_shop = [mariang, blueming, hera, rabbit,sharon,iamo]
string = ['마리앙로즈', '블루밍부케', '헤라플레르', '토끼풀꽃','샤론꽃집','이아모']

In [36]:
for i in range(len(hash_shop)):
    print('----------- %s -------------' % (string[i]))
    print(len(hash_shop[i]))

----------- 마리앙로즈 -------------
241
----------- 블루밍부케 -------------
246
----------- 헤라플레르 -------------
54
----------- 토끼풀꽃 -------------
172
----------- 샤론꽃집 -------------
16
----------- 이아모 -------------
11


#### 해시태그를 쓴 계정 정보 보기

In [37]:
for i in range(len(hash_shop)):
    print('------------ %s -----------' % (string[i]))
    print(hash_shop[i]['username'].value_counts())

------------ 마리앙로즈 -----------
mariang_rose_    241
Name: username, dtype: int64
------------ 블루밍부케 -----------
blooming__bouquet    246
Name: username, dtype: int64
------------ 헤라플레르 -----------
herafleur    54
Name: username, dtype: int64
------------ 토끼풀꽃 -----------
cloverflower_hongdae    171
kkot_zi                   1
Name: username, dtype: int64
------------ 샤론꽃집 -----------
sharon_flower1    16
Name: username, dtype: int64
------------ 이아모 -----------
iamo1991_official    10
forroom_byhee         1
Name: username, dtype: int64


#### 해시태그가 들어있는 게시글의 좋아요수 평균

In [38]:
# 좋아요수
for i in range(len(hash_shop)):
    print('----------- %s ------------' % (string[i]))
    print(hash_shop[i][['comment_count','like_count']].describe())

----------- 마리앙로즈 ------------
       comment_count  like_count
count     241.000000  241.000000
mean        0.580913  125.033195
std         1.644020   19.625848
min         0.000000  102.000000
25%         0.000000  113.000000
50%         0.000000  122.000000
75%         0.000000  131.000000
max        15.000000  323.000000
----------- 블루밍부케 ------------
       comment_count   like_count
count     246.000000   246.000000
mean        1.723577   352.845528
std         3.667398   193.486192
min         0.000000   142.000000
25%         0.000000   239.000000
50%         1.000000   304.000000
75%         2.000000   392.750000
max        37.000000  1693.000000
----------- 헤라플레르 ------------
       comment_count  like_count
count      54.000000   54.000000
mean        0.333333  144.259259
std         0.777029   41.451579
min         0.000000  102.000000
25%         0.000000  125.250000
50%         0.000000  137.000000
75%         0.000000  150.750000
max         4.000000  324.000000
-------

## insta_flower 정리

- 1) 해시태그에 많이 언급된 네일업체 : 블루밍부케, 마리앙로즈, 토끼풀꽃, 헤라플레르, 샤론꽃집, 이아모
- 2) 위치정보에 많이 태그된 네일업체 : 마리앙로즈,블루밍부케플라워샵,샤론꽃집,꽃집화요일,이아모,샐리인더플라워
-                          
- 해시태그나 위치정보에 많이 언급 및 태그된것이 업체에서 자기자신을 직접 태그한것
- 마리앙로즈, 블루밍부케, 토끼풀꽃